In [74]:
# Checking the status of Neptune Database. "Healthy" means it is working correctly.
%status

{'status': 'healthy',
 'startTime': 'Mon May 02 12:28:01 UTC 2022',
 'dbEngineVersion': '1.1.1.0.R1',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.5.2'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'clusterQueryTimeoutInMs': '120000'}}

## Create the graph

In below graph we are creating a tweet database for three users "mafudge", "Shubh143", "agokhale". The objective is to visualize the tweets, retweets done by these users with appropriate hashtags

%%oc :- This way we can write Cypher queries in jupyter supported by Neptune

In [6]:
%%oc
CREATE (shubh143:User {handle: "shubh143", followers: 30})
CREATE (agokhale:User {handle: "agokhale", followers: 40})
CREATE (mafudge:User {handle: "mafudge", followers: 1000})


CREATE (tweet1:Tweet {text: "As part of your latest, but it backend, databases, image generation. And proudly so Thanks for 222tez. #database", likes: 3 })
CREATE (tweet2:Tweet {text: "Ooooh, interesting! Just donated thanks In that insta-caches bitmaps and rarity scores are more. #awesome", likes: 23 })
CREATE (tweet3:Tweet {text: "Provides a lot of this Wednesday 27th of a few weeks, will be the animation, which makes sense of it. @mafudge #bethere #awesome", likes: 35 })
CREATE (tweet4:Tweet {text: "Compared to mint: You can recognize something : Heres a computer scientists do. And I could make a? #haha", likes: 78 })
CREATE (tweet5:Tweet {text: "Collected some months, but the shout out in my first still… I'm going strong : Collected My pleasure? #awesome", likes: 30 })
CREATE (tweet6:Tweet {text: "I'm still running and rarity scores are more general than that. Art, yes. But I also attended collector? #database #haha #awesome", likes: 31 })
CREATE (tweet7:Tweet {text: "Also, spotted this unique artwork to t… Roughly the grand opening? : I'm still disabled, opens 17:00! #bethere", likes: 98 })
CREATE (tweet8:Tweet {text: "Yeah I am Herbert and your iterative development and more professio… Thank you very much! Cool idea of? #database #haha", likes: 76 })
CREATE (tweet9:Tweet {text: "Thank you all! You can now see Fire Cards changed hands yesterday. Median 120tz, 2 last editions will be. #awesome", likes: 44 })
CREATE (tweet10:Tweet {text: "You can find them b… Ooooh, interesting! Just added some small fixes to go and Flash. Suddenly I. #haha #awesome", likes: 11 })

CREATE (haha:HASHTAG {hashtag: "#haha"})
CREATE (awesome:HASHTAG {hashtag: "#awesome"})
CREATE (bethere:HASHTAG {hashtag: "#bethere"})
CREATE (database:HASHTAG {hashtag: "#database"})

CREATE (shubh143)-[:TWEETED {date: "03/14/2022"}]-> (tweet1)
CREATE (shubh143)-[:TWEETED {date: "01/19/2022"}]-> (tweet2)
CREATE (shubh143)-[:TWEETED {date: "12/14/2021"}]-> (tweet3)
CREATE (agokhale)-[:TWEETED {date: "03/14/2022"}]-> (tweet4)
CREATE (agokhale)-[:TWEETED {date: "04/2/2022"}]-> (tweet5)
CREATE (agokhale)-[:TWEETED {date: "01/8/2022"}]-> (tweet6)
CREATE (agokhale)-[:TWEETED {date: "11/18/2021"}]-> (tweet7)
CREATE (mafudge)-[:TWEETED {date: "02/01/2021"}]-> (tweet8)
CREATE (mafudge)-[:TWEETED {date: "12/04/2021"}]-> (tweet9)
CREATE (mafudge)-[:TWEETED {date: "01/8/2022"}]-> (tweet10)

CREATE(mafudge)-[:RETWEETED {date: "01/09/2022"}] -> (tweet3)
CREATE (shubh143)-[:RETWEETED {date: "01/19/2022"}] -> (tweet9)
CREATE (agokhale)-[:RETWEETED {date: "09/19/2022"}] -> (tweet3)
CREATE (agokhale)-[:RETWEETED {date: "04/19/2022"}] -> (tweet10)

CREATE (tweet1)-[:HAS_HASHTAG]->(database)
CREATE (tweet2)-[:HAS_HASHTAG]->(awesome)
CREATE (tweet3)-[:HAS_HASHTAG]->(awesome)
CREATE (tweet3)-[:HAS_HASHTAG]->(bethere)
CREATE (tweet4)-[:HAS_HASHTAG]->(haha)
CREATE (tweet5)-[:HAS_HASHTAG]->(awesome)
CREATE (tweet6)-[:HAS_HASHTAG]->(database)
CREATE (tweet6)-[:HAS_HASHTAG]->(awesome)
CREATE (tweet6)-[:HAS_HASHTAG]->(haha)
CREATE (tweet7)-[:HAS_HASHTAG]->(bethere)
CREATE (tweet8)-[:HAS_HASHTAG]->(haha)
CREATE (tweet8)-[:HAS_HASHTAG]->(database)
CREATE (tweet9)-[:HAS_HASHTAG]->(awesome)
CREATE (tweet10)-[:HAS_HASHTAG]->(haha)
CREATE (tweet10)-[:HAS_HASHTAG]->(awesome)

CREATE (shubh143)-[:USED_HASHTAG]->(database)
CREATE (shubh143)-[:USED_HASHTAG]->(awesome)
CREATE (shubh143)-[:USED_HASHTAG]->(bethere)
CREATE (agokhale)-[:USED_HASHTAG]->(bethere)
CREATE (agokhale)-[:USED_HASHTAG]->(haha)
CREATE (agokhale)-[:USED_HASHTAG]->(awesome)
CREATE (agokhale)-[:USED_HASHTAG]->(database)
CREATE (mafudge)-[:USED_HASHTAG]->(database)
CREATE (mafudge)-[:USED_HASHTAG]->(awesome)
CREATE (mafudge)-[:USED_HASHTAG]->(haha)

CREATE (shubh143)-[:MENTIONED]->(mafudge)

Tab()

### Visualization of the graph

Running the next two cells will create a visualization of the entire tweeter graph. The cell immediately below is used to define a mapping between node labels and the property to be used to label nodes in the visualization.

In [72]:
display_var = '{"User":"handle","Tweet":"text","HASHTAG":"hashtag"}'

The query below creates a path containing the entire graph structure. The `-d` hint is used to enable the mappings defined above. The `-l` hint sets the maximum length of the text that can be displayed in a node. The text will be truncated 3 characters short of that length in cases where there is more text to display. An elipsis `...` is used to indicate that not all of the text was shown.

In [73]:
%%oc -d $display_var -l20
MATCH (n)-[r]->(d) RETURN n, r, d

Tab()

## Explore the relationships in the graph 

In the below code we are understanding tweets done by uer mafudge with #awesome hashtag

In [75]:
%%oc

match (u:User {handle:'mafudge'})-->(TWEETED)-->(HAS_HASHTAG {hashtag:"#awesome"})
return u.handle as Handle_Name, TWEETED.text as Tweet, HAS_HASHTAG.hashtag as Hashtag

Tab()

In the below code we are understanding follower counts of users greater than 30 which are using #awesome hashtags in their tweets

In [76]:
%%oc

match (u:User)-->(HAS_HASHTAG {hashtag:"#awesome"})
where u.followers > 30
return u.handle as Handle_Name, HAS_HASHTAG.hashtag as Hashtag

Tab()

Exploring the search option in the output console to analyze the use of specific keyword in our case '-thanks'

In [77]:
%%oc

match (u:User),(t:Tweet)
return u.handle, t.text

Tab()